In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGAN/200/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGAN/200/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000938', 'ENSG00000005844', 'ENSG00000009790',
       'ENSG00000013441', 'ENSG00000019582', 'ENSG00000028137',
       'ENSG00000038427', 'ENSG00000044574', 'ENSG00000049860',
       'ENSG00000051523',
       ...
       'ENSG00000204287', 'ENSG00000211772', 'ENSG00000211899',
       'ENSG00000223553', 'ENSG00000231500', 'ENSG00000240356',
       'ENSG00000244734', 'ENSG00000257207', 'ENSG00000269028', 'label'],
      dtype='object', length=115)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000938,ENSG00000005844,ENSG00000009790,ENSG00000013441,ENSG00000019582,ENSG00000028137,ENSG00000038427,ENSG00000044574,ENSG00000049860,ENSG00000051523,...,ENSG00000204287,ENSG00000211772,ENSG00000211899,ENSG00000223553,ENSG00000231500,ENSG00000240356,ENSG00000244734,ENSG00000257207,ENSG00000269028,label
VUMC-HC-0033-TR2591,4.432779,4.211676,4.252125,4.328301,7.136091,3.920710,3.720018,4.252125,4.432779,4.643048,...,5.318681,4.843824,4.911210,5.112603,6.828857,3.535280,6.995484,9.847484,7.757440,HC
Vumc-HD-70-TR1062,5.786592,5.247890,5.983595,5.114879,9.178061,4.546459,3.951486,5.653279,5.505492,6.338547,...,7.435968,6.744823,6.633504,3.522177,9.405474,3.701701,10.021953,8.262227,6.645036,HC
VUMC-HC0053-DOT-HD-48h-TR3087,5.526849,4.934939,5.442502,4.467036,7.631874,4.292393,3.535847,5.114817,5.038021,5.593316,...,5.631686,5.894780,4.559495,4.030369,7.399434,7.783065,7.050183,8.831693,4.253528,HC
Vumc-HD-149-TR932,4.923042,3.937986,4.556107,4.275620,6.897237,3.808559,3.647372,4.492627,3.083092,4.853377,...,5.606988,5.466326,4.389800,5.839804,8.609324,9.503594,8.661739,3.647372,8.517492,HC
Vumc-HD-36-TR1165,6.484772,5.938859,7.417415,6.564767,8.796382,5.339481,5.020424,3.862107,4.172174,7.306126,...,6.780833,7.808699,6.525333,3.862107,10.296126,5.671874,9.877834,3.862107,7.025844,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,6.691049,6.694907,5.825562,5.495377,7.600489,5.905014,5.933713,5.508326,5.401839,7.153655,...,6.681906,7.166033,6.141217,5.035161,8.366515,7.793853,9.116003,8.127146,6.462116,OC
77,4.354666,4.297767,5.332055,3.922808,7.127859,3.664713,3.860222,6.622599,4.438095,6.386422,...,5.556313,4.736933,3.839032,4.615925,7.203377,6.898293,9.982853,7.545402,5.018657,OC
78,5.041905,4.899195,3.914834,6.122008,8.971877,4.311927,4.261833,6.283450,4.732278,6.467687,...,4.911525,5.143180,4.729194,4.443622,7.080185,6.708379,8.443563,6.918291,7.780708,OC
79,7.501850,5.986558,6.251594,6.060142,6.950274,6.827371,7.576991,5.390357,4.792353,5.727593,...,9.226116,6.065059,8.448375,5.613737,8.268974,7.362110,7.770755,8.407131,6.862170,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

          HC       0.91      0.91      0.91       131
          OC       0.78      0.78      0.78        54

    accuracy                           0.87       185
   macro avg       0.84      0.84      0.84       185
weighted avg       0.87      0.87      0.87       185

[[119  12]
 [ 12  42]]
Dokładność: 0.8864864864864865
              precision    recall  f1-score   support

          HC       0.91      0.93      0.92       131
          OC       0.82      0.78      0.80        54

    accuracy                           0.89       185
   macro avg       0.87      0.85      0.86       185
weighted avg       0.89      0.89      0.89       185

[[122   9]
 [ 12  42]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7891891891891892
              precision    recall  f1-score   support

          HC       0.82      0.89      0.86       131
          OC       0.67      0.54      0.60        54

    accuracy                           0.79       185
   macro avg       0.75      0.72      0.73       185
weighted avg       0.78      0.79      0.78       185

[[117  14]
 [ 25  29]]
Dokładność: 0.8054054054054054
              precision    recall  f1-score   support

          HC       0.84      0.90      0.87       131
          OC       0.70      0.57      0.63        54

    accuracy                           0.81       185
   macro avg       0.77      0.74      0.75       185
weighted avg       0.80      0.81      0.80       185

[[118  13]
 [ 23  31]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8540540540540541
              precision    recall  f1-score   support

          HC       0.84      0.98      0.91       131
          OC       0.94      0.54      0.68        54

    accuracy                           0.85       185
   macro avg       0.89      0.76      0.79       185
weighted avg       0.87      0.85      0.84       185

[[129   2]
 [ 25  29]]
Dokładność: 0.8324324324324325
              precision    recall  f1-score   support

          HC       0.82      0.98      0.89       131
          OC       0.90      0.48      0.63        54

    accuracy                           0.83       185
   macro avg       0.86      0.73      0.76       185
weighted avg       0.84      0.83      0.81       185

[[128   3]
 [ 28  26]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       131
           1       0.83      0.70      0.76        54

    accuracy                           0.87       185
   macro avg       0.86      0.82      0.84       185
weighted avg       0.87      0.87      0.87       185

[[123   8]
 [ 16  38]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

           0       0.86      0.95      0.91       131
           1       0.85      0.63      0.72        54

    accuracy                           0.86       185
   macro avg       0.86      0.79      0.81       185
weighted avg       0.86      0.86      0.85       185

[[125   6]
 [ 20  34]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.8864864864864865
              precision    recall  f1-score   support

          HC       0.90      0.95      0.92       131
          OC       0.85      0.74      0.79        54

    accuracy                           0.89       185
   macro avg       0.87      0.84      0.86       185
weighted avg       0.88      0.89      0.88       185

[[124   7]
 [ 14  40]]
Dokładność: 0.8810810810810811
              precision    recall  f1-score   support

          HC       0.90      0.93      0.92       131
          OC       0.82      0.76      0.79        54

    accuracy                           0.88       185
   macro avg       0.86      0.85      0.85       185
weighted avg       0.88      0.88      0.88       185

[[122   9]
 [ 13  41]]
